<a href="https://colab.research.google.com/github/lagom-QB/M11/blob/master/Practice_10_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segmentation

In [0]:
from torchvision import models
from torch import nn
import torch
import torch.nn.functional as F
from pathlib import Path
import cv2
from torchvision import transforms
import numpy as np

In [0]:
class Identity(nn.Module):
  def forward(self, x):
    return x

In [0]:
class FlatChannel(nn.Module):
  def forward(self, x):
    batch_size = x.shape[0]
    width = x.shape[2]
    height = x.shape[3]
    return x.view(batch_size, x.shape[2], x.shape[3])

In [0]:
class StdUpsample(nn.Module):
    def __init__(self, nin, nout):
        super().__init__()
        self.conv = nn.ConvTranspose2d(nin, nout, kernel_size=2, stride=2)
        self.bn = nn.BatchNorm2d(nout)
        
    def forward(self, x): return self.bn(F.relu(self.conv(x)))

In [0]:
def upsample_model():
  return nn.Sequential(
    nn.ReLU(),
    StdUpsample(512,256),
    StdUpsample(256,128),
    StdUpsample(128,64),
    StdUpsample(64,64),
    nn.ConvTranspose2d(64, 1, kernel_size=2, stride=2),
    FlatChannel()
  )

In [0]:
def fconv():
  backbone =  models.resnet18(pretrained=True)
  downsample = nn.Sequential(backbone.conv1, 
                             backbone.bn1, 
                             backbone.relu, 
                             backbone.maxpool,
                             backbone.layer1 ,
                             backbone.layer2,
                             backbone.layer3 ,
                             backbone.layer4)
  upsample = upsample_model()
  return nn.Sequential(downsample, upsample)

In [0]:
class MyUnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = models.resnet18(pretrained=True)

    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)
        l1 = self.backbone.layer1(x)
        l2 = self.backbone.layer2(l1)
        l3 = self.backbone.layer3(l2)
        l4 = self.backbone.layer4(l3)
        x = nn.ReLU()(l4)
        x = StdUpsample(512,256)(x)
        x = torch.cat((l3, x), 1)
        x = StdUpsample(512,128)(x)
        x = torch.cat((l2, x), 1)
        x = StdUpsample(256,64)(x)
        x = torch.cat((l1, x), 1)
        x = StdUpsample(128,64)(x)
        x = nn.ConvTranspose2d(64, 1, kernel_size=2, stride=2)(x)
        x = FlatChannel()(x)

        return x

In [0]:
model = fconv()
model_1 = MyUnet()

In [11]:
model

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [12]:
model_1

MyUnet(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_run

In [0]:
test = torch.ones(4, 3, 64, 64)

In [14]:
model(test).shape

torch.Size([4, 64, 64])

In [15]:
!wget http://pages.cs.wisc.edu/~lizhang/projects/face-parsing/SmithCVPR2013_dataset_resized.zip

--2020-06-01 10:09:12--  http://pages.cs.wisc.edu/~lizhang/projects/face-parsing/SmithCVPR2013_dataset_resized.zip
Resolving pages.cs.wisc.edu (pages.cs.wisc.edu)... 128.105.7.26
Connecting to pages.cs.wisc.edu (pages.cs.wisc.edu)|128.105.7.26|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358268885 (342M) [application/zip]
Saving to: ‘SmithCVPR2013_dataset_resized.zip.1’

 SmithCVPR2013_data  80%[===============>    ] 274.54M  3.35MB/s    eta 24s    ^C


In [16]:
!unzip SmithCVPR2013_dataset_resized.zip

Archive:  SmithCVPR2013_dataset_resized.zip
replace SmithCVPR2013_dataset_resized/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
class FaceDataset(torch.utils.data.Dataset):
  def __init__(self, path: Path, transform):
    self._images_path = path / "images"
    self._labels_path = path / "labels"
    self._stems = list(map(lambda x: x.stem, self._images_path.glob("*")))
    self._transform = transform
    super().__init__()

  def __len__(self):
    return len(self._stems)
  
  def __getitem__(self, index: int):
    stem = self._stems[index]
    image_path = self._images_path / f"{stem}.jpg"
    label_path = self._labels_path / stem / f"{stem}_lbl00.png"
    image = cv2.imread(str(image_path))[:,:,::-1]
    label = 1 - cv2.imread(str(label_path), cv2.IMREAD_GRAYSCALE) // 255
    return self._transform(image, label)

In [0]:
class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, label=None):
        for t in self.transforms:
            image, label = t(image, label)
        return image, label

In [0]:
class Resize(object):
  def __init__(self, size=64):
    self.size = size

  def __call__(self, image, label=None):
    image = cv2.resize(image, (self.size,
                               self.size))
    if label is not None:
      label = cv2.resize(label, (self.size,
                                 self.size))
    return image, label

In [0]:
class Normalize(object):
  def __init__(self, mean = 127, std = 80):
    self.mean = mean
    self.std = std

  def __call__(self, image, label=None):
    return (image - self.mean) / self.std, label


In [0]:
class ToTensor(object):
    def __call__(self, image, labels=None):
      image = torch.from_numpy(image.astype(np.float32)).permute(2, 0, 1)
      if labels is not None:
        labels = torch.from_numpy(labels.astype(np.float32))
      return image, labels

In [0]:
transform = Compose([Resize(64),
                     Normalize(127, 80),
                     ToTensor()])

In [0]:
dataset = FaceDataset(Path("SmithCVPR2013_dataset_resized"), transform)

In [0]:
test_size = len(dataset) // 10

In [0]:
test_dataset, train_dataset = torch.utils.data.random_split(dataset, [test_size, len(dataset) - test_size])

In [0]:
loss = nn.BCEWithLogitsLoss()

In [0]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=True)

In [0]:
images, labels = train_loader.__iter__().next()

In [29]:
images.shape

torch.Size([256, 3, 64, 64])

In [30]:
labels.shape

torch.Size([256, 64, 64])

In [0]:
prediction = model(images)

In [32]:
prediction.shape

torch.Size([256, 64, 64])

In [33]:
loss(prediction, labels)

tensor(1.2082, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [0]:
def train(model, train_loader, optimizer, loss_function, epoch, writer = None):
    model.train()
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        if batch_idx % 2 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    train_loss /= len(train_loader.dataset)
    if writer is not None:
      writer.add_scalar("train_loss", train_loss, global_step=epoch)

def test(model, test_loader, loss_function, epoch, writer = None):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += loss_function(output, target).sum().item()
            pred = output.argmax(dim=1, keepdim=True)

    test_loss /= len(test_loader.dataset)
    if writer is not None:
      writer.add_scalar("test_loss", test_loss, global_step=epoch)
      writer.add_scalar("accuracy", accuracy, global_step=epoch)

    print('\nTest set: Average loss: {:.4f}'.format(
        test_loss, ))

In [0]:
optim = torch.optim.Adam(model.parameters())
for epoch in range(0, 10):
  train(model, train_loader, optim, loss, epoch)
  test(model, test_loader, loss, epoch)

Train Epoch: 0 [0/2097 (0%)]	Loss: 0.640602
Train Epoch: 0 [512/2097 (22%)]	Loss: 0.554216
Train Epoch: 0 [1024/2097 (44%)]	Loss: 0.493920
Train Epoch: 0 [1536/2097 (67%)]	Loss: 0.443289
Train Epoch: 0 [392/2097 (89%)]	Loss: 0.410379

Test set: Average loss: 0.0025
Train Epoch: 1 [0/2097 (0%)]	Loss: 0.386082
Train Epoch: 1 [512/2097 (22%)]	Loss: 0.389395
Train Epoch: 1 [1024/2097 (44%)]	Loss: 0.374355
Train Epoch: 1 [1536/2097 (67%)]	Loss: 0.368456
Train Epoch: 1 [392/2097 (89%)]	Loss: 0.338414

Test set: Average loss: 0.0024
Train Epoch: 2 [0/2097 (0%)]	Loss: 0.320732
Train Epoch: 2 [512/2097 (22%)]	Loss: 0.319503
Train Epoch: 2 [1024/2097 (44%)]	Loss: 0.308412
Train Epoch: 2 [1536/2097 (67%)]	Loss: 0.297999
Train Epoch: 2 [392/2097 (89%)]	Loss: 0.358973

Test set: Average loss: 0.0016
Train Epoch: 3 [0/2097 (0%)]	Loss: 0.276332
Train Epoch: 3 [512/2097 (22%)]	Loss: 0.268600
Train Epoch: 3 [1024/2097 (44%)]	Loss: 0.274745
Train Epoch: 3 [1536/2097 (67%)]	Loss: 0.263307
Train Epoch: 3 

In [1]:
optim = torch.optim.Adam(model_1.parameters())
for epoch in range(0, 10):
  train(model, train_loader, optim, loss, epoch)
  test(model, test_loader, loss, epoch)

NameError: ignored

In [0]:
test_images, test_labels = test_loader.__iter__().next()

In [0]:
prediction = torch.nn.Sigmoid()(model(test_images))

In [0]:
prediction.detach().numpy()

In [0]:
toPIL = transforms.ToPILImage()

In [0]:
toPIL(test_labels[3])

In [0]:
toPIL(prediction[3])